# PHYS4015 Lab 1

In [1]:
from __future__ import print_function
import torch
import os
import random
import numpy as np
import argparse
import scipy.io as sio
import math

import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.parallel

## 1.1 Tensors

### 1.1.1 Data objects as torch.tensor

In [2]:
data = [[1,2],[3,4]]
x_data = torch.tensor(data)
x_data

tensor([[1, 2],
        [3, 4]])

In [3]:
type(x_data)

torch.Tensor

In [4]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
type(x_np)

torch.Tensor

### 1.1.2 Random number generators

In [ ]:
np.ones()

In [5]:
x_ones = torch.ones_like(x_data)
print(f"Ones Tensor: \n {x_ones} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 



In [6]:
x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"Random Tensor: \n {x_rand} \n")

Random Tensor: 
 tensor([[0.3433, 0.1685],
        [0.2095, 0.2066]]) 



In [7]:
# shapes
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

In [8]:
rand_tensor.shape

torch.Size([2, 3])

In [9]:
# shapes
shape = (2,3,4)
rand_tensor = torch.rand(shape)

In [10]:
print(rand_tensor)

tensor([[[0.0753, 0.6563, 0.4151, 0.2865],
         [0.6873, 0.6049, 0.2861, 0.1231],
         [0.7049, 0.3586, 0.3321, 0.1129]],

        [[0.0841, 0.3433, 0.7291, 0.7803],
         [0.9026, 0.0331, 0.1987, 0.9326],
         [0.1192, 0.4779, 0.9468, 0.8188]]])


In [11]:
rand_tensor.shape[1]

3

### 1.1.3 Tensor operations

In [12]:
# tensor operations
tensor = torch.ones(4,4)
tensor[:,1] = 0
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [13]:
t1 = torch.cat([tensor, tensor, tensor], dim = 1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


In [14]:
# matrix multiplication
a = torch.ones((1,4))
b = torch.ones((4,4))
c = torch.mm(a,b)
print(c)

tensor([[4., 4., 4., 4.]])


In [15]:
# matrix multiplication
a = torch.ones((4,))
b = torch.ones((4,4))

In [16]:
torch.matmul(a,b)
# a @ b

tensor([4., 4., 4., 4.])

In [17]:
a.T

tensor([1., 1., 1., 1.])

In [18]:
a + a

tensor([2., 2., 2., 2.])

In [19]:
n = np.ones(5)
t = torch.from_numpy(n)
print(t)

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


## 1.2 Autograd

### 1.2.1 Inputs and Outputs

In [20]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1,3,64,64)
labels = torch.rand(1,1000)

In [21]:
model.state_dict

<bound method Module.state_dict of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [22]:
# output of the network corredponding to the input
prediction = model(data)
prediction

c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor([[-0.5030, -0.2340, -0.5639, -1.5148, -0.6230,  0.0814, -0.5804,  0.2441,
          0.2233, -0.8696, -1.0280, -1.0076, -0.3329, -0.8329, -1.3373, -0.6137,
         -0.8142, -0.4510, -0.8234, -0.4968, -1.5436, -0.9320, -1.3844, -0.0749,
         -1.1843, -1.4563, -0.8158, -0.9225, -1.0206, -0.2020, -0.8342, -0.6204,
         -0.4199, -0.6239, -0.4610, -0.5273,  0.3549, -0.9955, -0.4208,  0.2968,
         -0.6830, -0.8550, -1.1449, -0.1389, -0.5927, -0.6046, -0.8142, -0.4349,
         -1.0055, -1.3128, -0.7527,  0.4317, -0.4421, -0.7455, -0.2577, -1.1418,
         -0.4600, -1.5786, -0.7691, -0.4451,  0.6382,  0.0114, -0.1962,  0.1521,
         -0.8537, -0.3551, -0.4125, -0.4927, -0.9104, -0.9867, -1.5906,  0.2516,
         -1.6692, -0.3279, -1.2900, -1.5682, -0.0254, -0.7399,  0.2676, -0.1971,
         -0.9884, -1.6603, -0.1184, -0.9318, -0.7200, -0.1097,  0.0592,  0.3678,
         -0.2632, -0.8555, -1.3695, -1.3130, -2.1402, -0.5195, -0.0475, -2.5156,
         -0.5738, -0.5164, -

In [42]:
prediction.shape

torch.Size([1, 1000])

### 1.2.2 Gradient computation

In order to compute gradients or differentiate, some sort of computation graph must be contructed: https://pytorch.org/blog/computational-graphs-constructed-in-pytorch/

In [23]:
# define the loss function
loss = (prediction - labels).sum()
loss.backward()

In [24]:
# select an optimizer: https://pytorch.org/docs/stable/optim.html
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [25]:
optim.step()

In [26]:
weights = [p for p in model.parameters()]
len(weights)

62

In [27]:
# differentiation in autograd
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)


In [28]:
Q = 3*a**3 - b**2

In [29]:
external_grad = torch.tensor([1.,1.])
Q.backward(gradient=external_grad)

In [30]:
a.grad

tensor([36., 81.])

In [31]:
print(9*a**2 == a.grad)

tensor([True, True])


In [32]:
print(-2*b == b.grad)

tensor([True, True])


Once you specify `torch.no_grad()`, the gradients are no longer able to be computed.

In [33]:
with torch.no_grad():
    Q = 3*a**3 - b**2

external_grad = torch.tensor([1.,1.])
Q.backward(gradient=external_grad)

a.grad

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

## 1.3 Network Architecture

In [34]:
import os
import sys
from path_names import nn_path  # create your own DNN
# add full path of nn-train
#path = "c:\\somethingsomethingsomething\\nn-train"
path = nn_path
sys.path.append(path)

In [35]:
from train_stuff.model_loader import load

In [36]:
net = load('fc2')
print(net)

FullyConnected(
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=False)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=100, out_features=10, bias=False)
  )
)


In [46]:
net.state_dict().keys()

odict_keys(['fc.0.weight', 'fc.2.weight'])

In [37]:
weights = net.state_dict()
layers = list(net.state_dict().keys())
layers[1]
 

'fc.2.weight'

In [48]:
weights[layers[1]].shape

torch.Size([10, 100])

In [39]:
layers

['fc.0.weight', 'fc.2.weight']

In [40]:
weights

OrderedDict([('fc.0.weight',
              tensor([[ 0.0092, -0.0083,  0.0165,  ...,  0.0259,  0.0095, -0.0355],
                      [ 0.0050, -0.0188,  0.0099,  ...,  0.0250, -0.0327, -0.0160],
                      [-0.0102,  0.0046,  0.0077,  ..., -0.0342, -0.0055,  0.0088],
                      ...,
                      [-0.0205, -0.0198, -0.0065,  ...,  0.0148,  0.0088, -0.0151],
                      [-0.0338, -0.0051,  0.0066,  ...,  0.0207,  0.0257, -0.0117],
                      [-0.0011,  0.0324,  0.0229,  ..., -0.0104, -0.0246, -0.0247]])),
             ('fc.2.weight',
              tensor([[-0.0839,  0.0400, -0.0552, -0.0988, -0.0622, -0.0860, -0.0852,  0.0465,
                       -0.0417,  0.0708, -0.0899, -0.0161, -0.0810,  0.0362,  0.0525,  0.0594,
                       -0.0542,  0.0795,  0.0175, -0.0974, -0.0679,  0.0938, -0.0638, -0.0305,
                        0.0792,  0.0588, -0.0396, -0.0280, -0.0620, -0.0228,  0.0621,  0.0878,
                       -0.03

In [41]:
input = torch.ones(1,784)
net(input)

tensor([[ 0.0638,  0.0026,  0.2442, -0.1091,  0.0821, -0.0672,  0.2867, -0.0275,
         -0.1573,  0.1941]], grad_fn=<MmBackward>)

## References

1. https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

2. https://github.com/CKQu1/extended-criticality-dnn

3. Deep Learning, Goodfellow, I., Bengio, Y., Courville A.

4. Programming PyTorch for Deep Learning, Pointer, I.